In [1]:
#%matplotlib inline 
import torch 
#from IPython import display 
#from matplotlib import pyplot as plt
import numpy as np 
import random
import math
from time import time
#from torch import optim

In [2]:
def generatedata(noise,showpermutation=False,showtrue_w=False):
    true_w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if showtrue_w:
        print('true_w2:',true_w2)
    X2_before_ =torch.from_numpy(np.random.normal(0, 1, (num_example, num_X2feature)))
    y_ = torch.mm(X2_before_,true_w2)
    y_ += torch.from_numpy(np.random.normal(0, noise ,size=y_.size()))
    P_array=np.random.permutation(num_example)
    P=torch.zeros(num_example,num_example,dtype=torch.float64)
    for i in range(num_example):
        P[i][P_array[i]]=1
    if showpermutation:
        print('打乱X2的置换矩阵为',P)
    X2_=torch.mm(P,X2_before_)
    #X2_=X2_before_
    error_reg=(torch.norm(y_-torch.mm(X2_before_,true_w2))/torch.norm(y_))
    return y_,X2_,true_w2,P,error_reg

In [3]:
def generateinitialw(method,showinitialw=False):
    if method=='normal':
        w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if method=='zeros':
        w2=torch.zeros(num_X2feature,1,dtype=torch.float64)
    if showinitialw:
        print('initial w2:',w2)
    return w2

In [4]:
def sinkhorn_stabilized(a, b, M, reg, numItermax=1000, tau=1e3, stopThr=1e-9,
                        warmstart=None, verbose=False, print_period=20,
                        log=False, **kwargs):

#     a = np.asarray(a, dtype=np.float64)
#     b = np.asarray(b, dtype=np.float64)
#     M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M

#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # test if multiple target
#     if len(b.shape) > 1:
#         n_hists = b.shape[1]
#         a = a[:, np.newaxis]
#     else:
#         n_hists = 0
    n_hists = 0
    # init data
    dim_a = len(a)
    dim_b = len(b)

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    if n_hists:
        u = torch.ones((dim_a, n_hists)) / dim_a
        v = torch.ones((dim_b, n_hists)) / dim_b
    else:
        u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))- beta.reshape((1, dim_b))) / reg)

    def get_Gamma(alpha, beta, u, v):
        """log space gamma computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1)) - beta.reshape((1, dim_b)))
                      / reg + torch.log(u.reshape((dim_a, 1))) + torch.log(v.reshape((1, dim_b))))

    # print(np.min(K))

    K = get_K(alpha, beta)
    transp = K
    loop = 1
    cpt = 0
    err = 1
    while loop:

        uprev = u
        vprev = v
        # sinkhornrn update
        v = b / (torch.mm(K.transpose(1,0), u) + 1e-16)
        u = a / (torch.mm(K, v) + 1e-16)
        # remove numerical problems and store them in K
        if torch.abs(u).max() > tau or torch.abs(v).max() > tau:
            if n_hists:
                alpha, beta = alpha + reg * \
                    torch.max(torch.log(u), 1), beta + reg * torch.max(np.log(v))
            else:
                alpha, beta = alpha + reg * torch.log(u), beta + reg * torch.log(v)
                if n_hists:
                    u, v = torch.ones((dim_a, n_hists)) / dim_a, torch.ones((dim_b, n_hists)) / dim_b
                else:
                    u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b
            K = get_K(alpha, beta)
            

        if cpt % print_period == 0:
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            if n_hists:
                err_u = abs(u - uprev).max()
                err_u /= max(abs(u).max(), abs(uprev).max(), 1.)
                err_v = abs(v - vprev).max()
                err_v /= max(abs(v).max(), abs(vprev).max(), 1.)
                err = 0.5 * (err_u + err_v)
            else:
                transp = get_Gamma(alpha, beta, u, v)
                err = torch.norm((torch.sum(transp, axis=0) - b))
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 20) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr:
            loop = False

        if cpt >= numItermax:
            loop = False

        if np.any(np.isnan(u.detach().numpy())) or np.any(np.isnan(v.detach().numpy())):
            # we have reached the machine precision
            # come back to previous solution and quit loop
            print('Warning: numerical errors at iteration', cpt)
            u = uprev
            v = vprev
            break

        cpt = cpt + 1
    #print(cpt)
    if log:
        if n_hists:
            alpha = alpha[:, None]
            beta = beta[:, None]
        logu = alpha / reg + torch.log(u)
        logv = beta / reg + torch.log(v)
        log['logu'] = logu
        log['logv'] = logv
        log['alpha'] = alpha + reg * torch.log(u)
        log['beta'] = beta + reg * torch.log(v)
        log['warmstart'] = (log['alpha'], log['beta'])
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res, log

        else:
            return get_Gamma(alpha, beta, u, v), log
    else:
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res
        else:
            return get_Gamma(alpha, beta, u, v)

In [5]:
def sinkhorn_epsilon_scaling(a, b, M, reg, numItermax=100, epsilon0=1e4,
                             numInnerItermax=100, tau=1e3, stopThr=1e-9,
                             warmstart=None, verbose=False, print_period=10,
                             log=False, **kwargs):
    #a = np.asarray(a, dtype=np.float64)
    #b = np.asarray(b, dtype=np.float64)
    #M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M
#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # init data
    dim_a = len(a)
    #dim_a=num_example
    dim_b = len(b)
    #dim_b=num_example
    # nrelative umerical precision with 64 bits
    numItermin = 35
    numItermax = max(numItermin, numItermax)  # ensure that last velue is exact

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))
                        - beta.reshape((1, dim_b))) / reg)

    # print(np.min(K))
    def get_reg(n):  # exponential decreasing
        return (epsilon0 - reg) * np.exp(-n) + reg

    loop = 1
    cpt = 0
    err = 1
    while loop:

        regi = get_reg(cpt)

        G, logi = sinkhorn_stabilized(a, b, M, regi,
                                      numItermax=numInnerItermax, stopThr=1e-9,
                                      warmstart=(alpha, beta), verbose=False,
                                      print_period=20, tau=tau, log=True)

        alpha = logi['alpha']
        beta = logi['beta']

        if cpt >= numItermax:
            loop = False

        if cpt % (print_period) == 0:  # spsion nearly converged
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            transp = G
            err = torch.norm(
                (torch.sum(transp, axis=0) - b))**2 + torch.norm((torch.sum(transp, axis=1) - a))**2
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 10) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr and cpt > numItermin:
            loop = False

        cpt = cpt + 1
    # print('err=',err,' cpt=',cpt)
    if log:
        log['alpha'] = alpha
        log['beta'] = beta
        log['warmstart'] = (log['alpha'], log['beta'])
        return G, log
    else:
        return G
    

In [21]:
gama_=1
eta=0
starts=1
for i_____ in range(5):
    for num_example in range(200,201,100): 
        for num_X2feature in [30]:
            for i____ in range(10):
                (y_,X2_,true_w2,true_P,error_reg1)=generatedata(noise=0.3)
                y=y_
                X2=X2_
                results_Loss = []
                results_w2=[]
                results_error=[]
                for i__ in range(starts):
    #                     P_array=np.random.permutation(num_example)
    #                     P=torch.zeros(num_example,num_example,dtype=torch.float64)
    #                     for i in range(num_example):
    #                         P[i][P_array[i]]=1
    #                     X_=torch.cat([X1,X2],1)
    #                     X=torch.mm(P,X_)
    #                     w=torch.mm(torch.mm(torch.tensor(np.linalg.inv(torch.mm(X.transpose(1,0),X))),X.transpose(1,0)),y)
    #                     w1,w2=w.split([num_X1feature,num_X2feature],dim=0)
    #                     w1=torch.from_numpy(np.random.normal(0, 0,(num_X1feature,1)))
    #                     w2=torch.from_numpy(np.random.normal(0, 0,(num_X2feature,1)))
                    w2=generateinitialw(method='zeros')
                    #w2=true_w2
                    w2.requires_grad_(requires_grad=True)
    #                 results_Loss = []
                    lr=0.002
                    results_S=[]
                    t=0
                    before1=0
                    while True:
                        #start=time()
                        Y1=y
                        Y2=torch.mm(X2,w2)
                        C=torch.zeros(num_example,num_example,dtype=torch.float64)
                        for i in range(num_example):
                            for j in range(num_example):
                                C[i][j]=(Y1[i]-Y2[j])**2            

                        #S=SinkhornIPOT(C)
                        a=torch.ones(num_example,1,dtype=torch.float64)
                        b=torch.ones(num_example,1,dtype=torch.float64)
                        S=sinkhorn_epsilon_scaling(a, b, C, 0.00000001)
                        #print(S.transpose(1,0).half())
                        #results_S.append(S)
                        #if t>0:
                            #print('        S变化',(torch.norm(results_S[t]-results_S[t-1]))/(torch.norm(results_S[t-1])))
                        #Loss=torch.sum(S*C)
                        Loss=torch.norm(Y1-torch.mm(S,Y2))**2
                        if Loss<1e-2:
                            break
                        Loss.backward()
    #                         results_Loss.append(Loss)
    #                         for i_ in range(num_X1features):
    #                             results_w1[t][i_]=(w1[i_].data)
    #                         for i_ in range(num_X2features):
    #                             results_w2[t][i_]=(w2[i_].data)
                        w2.data-=lr*(w2.grad+np.random.normal(0,np.sqrt(eta/(1+t)**gama_)))
                        #print(w2.grad)
    #                     if t==num_epochs-1:
    #                         print('最终w1梯度：',w1.grad)
    #                         print('最终w2梯度：',w2.grad)
                        w2.grad.data.zero_() 
                        print('Loss',t,'=',Loss.data)
    #                     if t%6==0:
    #                         if torch.norm(Loss-before1)<1e-4:
    #                             break
    #                         before1=Loss
                        if torch.norm(Loss-before1)/before1<1e-7:
                            break
                        before1=Loss
                        if t>=200:
                            print('超过迭代上限')
                            break
                        if math.isnan(Loss):
                            break
                        t+=1
                        print('#',end='')
                        #print(time()-start)



                    print(' ',end='')
                    error_each=(torch.norm(w2-true_w2))
                    results_error.append(error_each)
                    #results_Loss.append(Loss)
                    #results_w1.append(w1.data)
                    #results_w2.append(w2.data)


                #w1=results_w1[results_Loss.index(min(results_Loss))]
                #w2=results_w2[results_Loss.index(min(results_Loss))]

    #                     for i_ in range(starts):
    #                         results_w1[i_]=(w1[i_].data)
    #                     for i_ in range(starts):
    #                         results_w2[i_]=(w2[i_].data)


                #error_w=((torch.norm(w1-true_w1))/(torch.norm(true_w1))+(torch.norm(w2-true_w2))/(torch.norm(true_w2)))/2
                #print(num_X1feature,num_X2feature,num_example,'平均相对误差1：',error_w)
                print(num_X2feature,num_example,'平均相对误差2：',np.min(results_error),end='   ')

                #print('真实置换矩阵为：',true_P)
                error_P=(torch.norm(S.transpose(1,0)-true_P))/(torch.norm(true_P))
                print(num_X2feature,num_example,'置换矩阵误差：',error_P,end='   ')
                error_reg2=(torch.norm(y_-torch.mm(torch.mm(S,X2_),w2))/torch.norm(y_))
                print('真实回归误差',error_reg1,end='   ')
                print('实验回归误差',error_reg2)
                #print('双随机矩阵S为：',S.transpose(1,0).half())
                #print(results)
    #                 plt.figure(figsize=(6,6))
    #                 plt.plot(results_w1_0,results_Loss, '-o',label='$w1[0]$')
    #                 plt.plot(results_w1_1,results_Loss, '-o',label='$w1[1]$')
    #                 plt.plot(results_w1_2,results_Loss, '-o',label='$w1[2]$')
    #                 plt.plot(results_w2_0,results_Loss, '-o',label='$w2[0]$')
    #                 plt.plot(results_w2_1,results_Loss, '-o',label='$w2[1]$')
    #                 plt.plot(results_w2_2,results_Loss, '-o',label='$w2[2]$')
    #                 plt.legend()
    #                 plt.show()

Loss 0 = tensor(4550.5347, dtype=torch.float64)
#Loss 1 = tensor(4309.7307, dtype=torch.float64)
#Loss 2 = tensor(307.3619, dtype=torch.float64)
#Loss 3 = tensor(280.0546, dtype=torch.float64)
#Loss 4 = tensor(257.9248, dtype=torch.float64)
#Loss 5 = tensor(236.4623, dtype=torch.float64)
#Loss 6 = tensor(213.7367, dtype=torch.float64)
#Loss 7 = tensor(191.2625, dtype=torch.float64)
#Loss 8 = tensor(172.6699, dtype=torch.float64)
#Loss 9 = tensor(160.4479, dtype=torch.float64)
#Loss 10 = tensor(150.5823, dtype=torch.float64)
#Loss 11 = tensor(140.0085, dtype=torch.float64)
#Loss 12 = tensor(125.0744, dtype=torch.float64)
#Loss 13 = tensor(109.6367, dtype=torch.float64)
#Loss 14 = tensor(96.4119, dtype=torch.float64)
#Loss 15 = tensor(85.4794, dtype=torch.float64)
#Loss 16 = tensor(76.5304, dtype=torch.float64)
#Loss 17 = tensor(69.7522, dtype=torch.float64)
#Loss 18 = tensor(63.7359, dtype=torch.float64)
#Loss 19 = tensor(58.5355, dtype=torch.float64)
#Loss 20 = tensor(54.5445, dtype=to

#Loss 6 = tensor(339.6749, dtype=torch.float64)
#Loss 7 = tensor(302.2414, dtype=torch.float64)
#Loss 8 = tensor(262.2560, dtype=torch.float64)
#Loss 9 = tensor(227.7355, dtype=torch.float64)
#Loss 10 = tensor(198.3672, dtype=torch.float64)
#Loss 11 = tensor(176.5266, dtype=torch.float64)
#Loss 12 = tensor(159.2620, dtype=torch.float64)
#Loss 13 = tensor(140.4073, dtype=torch.float64)
#Loss 14 = tensor(122.8987, dtype=torch.float64)
#Loss 15 = tensor(107.1409, dtype=torch.float64)
#Loss 16 = tensor(93.0576, dtype=torch.float64)
#Loss 17 = tensor(78.8840, dtype=torch.float64)
#Loss 18 = tensor(68.8196, dtype=torch.float64)
#Loss 19 = tensor(60.8158, dtype=torch.float64)
#Loss 20 = tensor(53.0163, dtype=torch.float64)
#Loss 21 = tensor(46.3991, dtype=torch.float64)
#Loss 22 = tensor(41.7802, dtype=torch.float64)
#Loss 23 = tensor(37.5955, dtype=torch.float64)
#Loss 24 = tensor(33.5008, dtype=torch.float64)
#Loss 25 = tensor(30.6050, dtype=torch.float64)
#Loss 26 = tensor(27.9925, dtype=t

#Loss 100 = tensor(10.7448, dtype=torch.float64)
#Loss 101 = tensor(10.7431, dtype=torch.float64)
#Loss 102 = tensor(10.7413, dtype=torch.float64)
#Loss 103 = tensor(10.7408, dtype=torch.float64)
#Loss 104 = tensor(10.7407, dtype=torch.float64)
#Loss 105 = tensor(10.7406, dtype=torch.float64)
#Loss 106 = tensor(10.7406, dtype=torch.float64)
#Loss 107 = tensor(10.7406, dtype=torch.float64)
#Loss 108 = tensor(10.7406, dtype=torch.float64)
#Loss 109 = tensor(10.7406, dtype=torch.float64)
#Loss 110 = tensor(10.7406, dtype=torch.float64)
#Loss 111 = tensor(10.7406, dtype=torch.float64)
#Loss 112 = tensor(10.7406, dtype=torch.float64)
 30 200 平均相对误差2： tensor(6.5031, dtype=torch.float64, grad_fn=<NormBackward0>)   30 200 置换矩阵误差： tensor(1.4142, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0517, dtype=torch.float64)   实验回归误差 tensor(0.0446, dtype=torch.float64, grad_fn=<DivBackward0>)
Loss 0 = tensor(2749.7328, dtype=torch.float64)
#Loss 1 = tensor(2568.3679, dtype=torch.float6

#Loss 64 = tensor(5.8291, dtype=torch.float64)
#Loss 65 = tensor(5.7428, dtype=torch.float64)
#Loss 66 = tensor(5.6665, dtype=torch.float64)
#Loss 67 = tensor(5.6066, dtype=torch.float64)
#Loss 68 = tensor(5.5441, dtype=torch.float64)
#Loss 69 = tensor(5.5130, dtype=torch.float64)
#Loss 70 = tensor(5.5009, dtype=torch.float64)
#Loss 71 = tensor(5.4956, dtype=torch.float64)
#Loss 72 = tensor(5.4790, dtype=torch.float64)
#Loss 73 = tensor(5.4316, dtype=torch.float64)
#Loss 74 = tensor(5.3336, dtype=torch.float64)
#Loss 75 = tensor(5.2750, dtype=torch.float64)
#Loss 76 = tensor(5.2109, dtype=torch.float64)
#Loss 77 = tensor(5.1794, dtype=torch.float64)
#Loss 78 = tensor(5.1533, dtype=torch.float64)
#Loss 79 = tensor(5.1274, dtype=torch.float64)
#Loss 80 = tensor(5.1186, dtype=torch.float64)
#Loss 81 = tensor(5.1102, dtype=torch.float64)
#Loss 82 = tensor(5.1023, dtype=torch.float64)
#Loss 83 = tensor(5.0890, dtype=torch.float64)
#Loss 84 = tensor(5.0779, dtype=torch.float64)
#Loss 85 = te

#Loss 40 = tensor(20.5504, dtype=torch.float64)
#Loss 41 = tensor(19.8181, dtype=torch.float64)
#Loss 42 = tensor(19.1680, dtype=torch.float64)
#Loss 43 = tensor(18.4176, dtype=torch.float64)
#Loss 44 = tensor(17.7378, dtype=torch.float64)
#Loss 45 = tensor(17.1053, dtype=torch.float64)
#Loss 46 = tensor(16.8445, dtype=torch.float64)
#Loss 47 = tensor(16.5250, dtype=torch.float64)
#Loss 48 = tensor(16.1603, dtype=torch.float64)
#Loss 49 = tensor(15.8630, dtype=torch.float64)
#Loss 50 = tensor(15.7083, dtype=torch.float64)
#Loss 51 = tensor(15.5546, dtype=torch.float64)
#Loss 52 = tensor(15.4320, dtype=torch.float64)
#Loss 53 = tensor(15.3461, dtype=torch.float64)
#Loss 54 = tensor(15.2870, dtype=torch.float64)
#Loss 55 = tensor(15.2034, dtype=torch.float64)
#Loss 56 = tensor(15.1501, dtype=torch.float64)
#Loss 57 = tensor(15.1338, dtype=torch.float64)
#Loss 58 = tensor(15.0720, dtype=torch.float64)
#Loss 59 = tensor(15.0073, dtype=torch.float64)
#Loss 60 = tensor(14.9395, dtype=torch.f

...

#Loss 17 = tensor(130.4510, dtype=torch.float64)
#Loss 18 = tensor(116.3011, dtype=torch.float64)
#Loss 19 = tensor(105.3894, dtype=torch.float64)
#Loss 20 = tensor(93.0047, dtype=torch.float64)
#Loss 21 = tensor(81.0441, dtype=torch.float64)
#Loss 22 = tensor(72.2397, dtype=torch.float64)
#Loss 23 = tensor(67.4017, dtype=torch.float64)
#Loss 24 = tensor(62.7147, dtype=torch.float64)
#Loss 25 = tensor(57.8892, dtype=torch.float64)
#Loss 26 = tensor(53.9289, dtype=torch.float64)
#Loss 27 = tensor(51.0080, dtype=torch.float64)
#Loss 28 = tensor(48.6153, dtype=torch.float64)
#Loss 29 = tensor(45.6988, dtype=torch.float64)
#Loss 30 = tensor(43.5749, dtype=torch.float64)
#Loss 31 = tensor(41.7611, dtype=torch.float64)
#Loss 32 = tensor(39.8969, dtype=torch.float64)
#Loss 33 = tensor(38.9083, dtype=torch.float64)
#Loss 34 = tensor(38.4222, dtype=torch.float64)
#Loss 35 = tensor(37.8135, dtype=torch.float64)
#Loss 36 = tensor(36.0986, dtype=torch.float64)
#Loss 37 = tensor(35.2484, dtype=torc

#Loss 50 = tensor(23.7966, dtype=torch.float64)
#Loss 51 = tensor(23.2925, dtype=torch.float64)
#Loss 52 = tensor(22.9474, dtype=torch.float64)
#Loss 53 = tensor(22.5435, dtype=torch.float64)
#Loss 54 = tensor(22.2454, dtype=torch.float64)
#Loss 55 = tensor(21.8765, dtype=torch.float64)
#Loss 56 = tensor(21.5187, dtype=torch.float64)
#Loss 57 = tensor(21.2711, dtype=torch.float64)
#Loss 58 = tensor(21.0270, dtype=torch.float64)
#Loss 59 = tensor(20.7387, dtype=torch.float64)
#Loss 60 = tensor(20.5824, dtype=torch.float64)
#Loss 61 = tensor(20.2468, dtype=torch.float64)
#Loss 62 = tensor(20.0154, dtype=torch.float64)
#Loss 63 = tensor(19.8037, dtype=torch.float64)
#Loss 64 = tensor(19.6113, dtype=torch.float64)
#Loss 65 = tensor(19.4747, dtype=torch.float64)
#Loss 66 = tensor(19.3485, dtype=torch.float64)
#Loss 67 = tensor(19.1188, dtype=torch.float64)
#Loss 68 = tensor(18.9059, dtype=torch.float64)
#Loss 69 = tensor(18.7156, dtype=torch.float64)
#Loss 70 = tensor(18.4041, dtype=torch.f

#Loss 95 = tensor(30.9815, dtype=torch.float64)
#Loss 96 = tensor(30.9232, dtype=torch.float64)
#Loss 97 = tensor(30.8479, dtype=torch.float64)
#Loss 98 = tensor(30.8118, dtype=torch.float64)
#Loss 99 = tensor(30.7849, dtype=torch.float64)
#Loss 100 = tensor(30.6455, dtype=torch.float64)
#Loss 101 = tensor(30.4991, dtype=torch.float64)
#Loss 102 = tensor(30.2536, dtype=torch.float64)
#Loss 103 = tensor(29.4034, dtype=torch.float64)
#Loss 104 = tensor(28.7252, dtype=torch.float64)
#Loss 105 = tensor(28.2312, dtype=torch.float64)
#Loss 106 = tensor(27.9872, dtype=torch.float64)
#Loss 107 = tensor(27.7968, dtype=torch.float64)
#Loss 108 = tensor(27.6221, dtype=torch.float64)
#Loss 109 = tensor(27.4781, dtype=torch.float64)
#Loss 110 = tensor(27.3118, dtype=torch.float64)
#Loss 111 = tensor(27.1326, dtype=torch.float64)
#Loss 112 = tensor(26.8680, dtype=torch.float64)
#Loss 113 = tensor(26.6661, dtype=torch.float64)
#Loss 114 = tensor(26.5509, dtype=torch.float64)
#Loss 115 = tensor(26.478

#Loss 30 = tensor(83.1189, dtype=torch.float64)
#Loss 31 = tensor(79.3875, dtype=torch.float64)
#Loss 32 = tensor(75.7960, dtype=torch.float64)
#Loss 33 = tensor(73.2712, dtype=torch.float64)
#Loss 34 = tensor(71.5051, dtype=torch.float64)
#Loss 35 = tensor(69.5742, dtype=torch.float64)
#Loss 36 = tensor(67.1960, dtype=torch.float64)
#Loss 37 = tensor(64.3345, dtype=torch.float64)
#Loss 38 = tensor(60.8467, dtype=torch.float64)
#Loss 39 = tensor(58.0045, dtype=torch.float64)
#Loss 40 = tensor(55.2456, dtype=torch.float64)
#Loss 41 = tensor(52.3715, dtype=torch.float64)
#Loss 42 = tensor(49.3146, dtype=torch.float64)
#Loss 43 = tensor(47.1141, dtype=torch.float64)
#Loss 44 = tensor(45.6249, dtype=torch.float64)
#Loss 45 = tensor(44.3064, dtype=torch.float64)
#Loss 46 = tensor(43.1791, dtype=torch.float64)
#Loss 47 = tensor(42.3259, dtype=torch.float64)
#Loss 48 = tensor(41.6173, dtype=torch.float64)
#Loss 49 = tensor(41.0253, dtype=torch.float64)
#Loss 50 = tensor(40.0484, dtype=torch.f

KeyboardInterrupt: 